In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

import altair as alt
alt.data_transformers.disable_max_rows()

import datetime
import seaborn as sns
import matplotlib.pyplot as plt

## Set constants

In [ ]:
latent_heat_of_sublimation_ice = 2838000 #J/kg
weight_cubic_meter_of_water = 1000 # kg/m^3

## Read in data

In [ ]:
flux_df = pd.read_parquet('../data/tilden_fluxdata_consolidated.parquet')

## Sort data by date and set NODATA values to NaN

In [ ]:
flux_df = flux_df.sort_values('date')

flux_df = flux_df.replace(-999, np.nan)

In [ ]:
flux_df

## Limit data to the 2021 snow-on season

In [ ]:
flux_df = flux_df[flux_df['date'] > datetime.datetime(2021, 12, 6)]
flux_df = flux_df[flux_df['date'] < datetime.datetime(2022, 4, 1)]

## Convert kelvin to celcius

In [ ]:
flux_df['Tair'] = flux_df['Tair'] - 273.15

## Plot diurnal cycle of sublimation using latent heat flux estimates from the sonics (eddy covariance measurements)

In [ ]:
src = flux_df.copy()

src = src.query("qc_LE == 0")
src = src.groupby([src['date'].dt.hour, 'height']).mean().reset_index() # latent heat flux in joules/m^2/sec AKA watts/m^2
src['sublimation_rate'] = src['LE'] / latent_heat_of_sublimation_ice # kg/m^2/sec
src['sublimation_rate'] = src['sublimation_rate'] / weight_cubic_meter_of_water # m/sec
src['sublimation_rate'] = src['sublimation_rate'] * 1000 * 3600  # mm/hr

alt.Chart(src).mark_line().encode(
    alt.X('date', title='Hour'),
    alt.Y('sublimation_rate', title='Sublimation/Deposition Rate (mm/hr)'),
    alt.Color('height:N')
)

## Plot diurnal cycle of wind direction

In [ ]:
src = flux_df.copy()
src = src.replace(-999, np.nan)
src['hour'] = src['date'].dt.hour

boxplot = alt.Chart(src).mark_boxplot(outliers={'size': 5}, color='lightgrey').encode(
    x='hour:O',
    y='wind_dir:Q'
)

src = src.groupby(src['date'].dt.hour).mean().reset_index() # latent heat flux in joules/m^2/sec AKA watts/m^2

lineplot = alt.Chart(src).mark_line().encode(
    alt.X('hour:O'),
    alt.Y('wind_dir:Q')
)

boxplot + lineplot

## Plot diurnal cycle of vertical wind variance

In [ ]:
src = flux_df.copy()
src = src.replace(-999, np.nan)
src['hour'] = src['date'].dt.hour

boxplot = alt.Chart(src).mark_boxplot(outliers={'size': 5}, color='lightgrey').encode(
    x='hour:O',
    y='w_unrot:Q'
)

boxplot_w_var = alt.Chart(src).mark_boxplot(outliers={'size': 5}, color='lightgrey').encode(
    x='hour:O',
    y='w_var:Q'
)

src = src.groupby(src['date'].dt.hour).mean().reset_index() # latent heat flux in joules/m^2/sec AKA watts/m^2

lineplot = alt.Chart(src).mark_line().encode(
    alt.X('hour:O'),
    alt.Y('w_unrot:Q')
)

lineplot_w_var = alt.Chart(src).mark_line().encode(
    alt.X('hour:O'),
    alt.Y('w_var:Q')
)

(boxplot + lineplot) | (boxplot_w_var + lineplot_w_var)

In [ ]:
def create_nighttime_dataframe(start_date, end_date, night_start_hour = 18, night_end_hour = 6):
    return pd.DataFrame({
        'start_date': [datetime.datetime(start_date.year, start_date.month, start_date.day, 0)] + [start_date + datetime.timedelta(days=i) + datetime.timedelta(hours=night_start_hour) for i in range(0, (end_date - start_date).days)],
        'end_date': [start_date + datetime.timedelta(days = i) + datetime.timedelta(hours = night_end_hour) for i in range(0, (end_date - start_date).days)] + [datetime.datetime(end_date.year, end_date.month, end_date.day, 0)]
    })

## Plot wind direction timeseries during a windy, generally clear period (February 10 - 13)

### Read in KPS radar data

In [ ]:
radar_winds_df = pd.read_pickle("kp_radar_windspeed.pickle")
radar_winds_df['wind_direction'] = radar_winds_df['wind_direction'].apply(lambda dir: dir-360 if dir > 180 else dir)

In [ ]:
radar_winds_df.height.unique()

In [ ]:
start_date = datetime.datetime(2022, 2, 6)
end_date = datetime.datetime(2022, 2, 14)

src = flux_df[flux_df['date'] > start_date]
src = src[src['date'] < end_date]
night_df = create_nighttime_dataframe(start_date, end_date, night_start_hour = 17, night_end_hour = 8)
src['wind_dir'] = src['wind_dir'].apply(lambda dir: dir-360 if dir > 180 else dir)

wind_dir_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).mark_point().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'wind_dir:Q',
            title='Wind Direction',
            scale=alt.Scale(domain=[-180, 180], nice=False), 
            axis=alt.Axis(
                values=[-180, -90, 0, 90, 180],
            )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Wind direction during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        width=1200
    ) + alt.Chart(radar_winds_df).transform_filter(alt.datum.height < 15).mark_point(size=25, color='black', strokeWidth=3).encode(
            alt.X('time:T'),
            alt.Y('wind_direction')
        )
)

In [ ]:
wind_speed_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).mark_point().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'wind_speed:Q',
            title='Wind Speed',
            # scale=alt.Scale(domain=[-180, 180], nice=False), 
            # axis=alt.Axis(
            #     values=[-180, -90, 0, 90, 180],
            # )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Wind Speed during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        width=1200
    ) + alt.Chart(radar_winds_df).transform_filter(alt.datum.height < 15).mark_point(size=25, color='black', strokeWidth=3).encode(
            alt.X('time:T'),
            alt.Y('wind_speed')
        )
)

In [ ]:
le_flux_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).transform_filter(alt.datum.height == 3).mark_line().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'LE:Q',
            title='Latent Heat Flux',
            # scale=alt.Scale(domain=[-180, 180], nice=False), 
            # axis=alt.Axis(
            #     values=[-180, -90, 0, 90, 180],
            # )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Latent Heat Flux during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        height=200,
        width=1200
    )
)

In [ ]:
rh_change_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).transform_filter(alt.datum.height == 3).mark_line().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'RH:Q',
            title='Relative Humidity',
            # scale=alt.Scale(domain=[-180, 180], nice=False), 
            # axis=alt.Axis(
            #     values=[-180, -90, 0, 90, 180],
            # )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Relative humidity during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        height=200,
        width=1200
    )
)

In [ ]:
w_unrot_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).transform_filter(alt.datum.height == 3).mark_line().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'w_unrot:Q',
            title='Vertical velocity',
            # scale=alt.Scale(domain=[-180, 180], nice=False), 
            # axis=alt.Axis(
            #     values=[-180, -90, 0, 90, 180],
            # )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Vertical velocity during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        height=200,
        width=1200
    )
)

In [ ]:
w_var_plot = (
    alt.Chart(night_df).mark_rect(color='grey', opacity=0.2).encode(
        alt.X('start_date:T'),
        alt.X2('end_date')
    ) + alt.Chart(src).transform_filter(alt.datum.height == 3).mark_line().encode(
        alt.X(
            'date:T', 
            title='Hour',
            axis = alt.Axis(format = ("%H")),
        ), 
        alt.Y(
            'w_var:Q',
            title='Vertical velocity variance',
            # scale=alt.Scale(domain=[-180, 180], nice=False), 
            # axis=alt.Axis(
            #     values=[-180, -90, 0, 90, 180],
            # )
        ),
        alt.Color('height:N')
    ).properties(
        title=f'Vertical velocity variance during a windy and clear period, {start_date.strftime("%m-%d")} through {end_date.strftime("%m-%d")}',
        height=200,
        width=1200
    )
)

In [ ]:
(wind_dir_plot & wind_speed_plot & le_flux_plot & rh_change_plot & w_var_plot & w_unrot_plot).display(renderer='svg')

## Explore radar wind profiles

In [ ]:
radar_winds_df['wind_speed_lower_bound'] = radar_winds_df['wind_speed'] - radar_winds_df['wind_speed_error']
radar_winds_df['wind_speed_upper_bound'] = radar_winds_df['wind_speed'] + radar_winds_df['wind_speed_error']

In [ ]:
radar_winds_df

In [ ]:
radar_wind_speed_profile_plots = (
    alt.Chart(radar_winds_df).transform_filter(
        alt.datum.height <= 200
    ).transform_filter(
        alt.datum.wind_speed_error <= 50
    ).mark_line().encode(
        alt.X('wind_speed', title='Wind Speed', scale=alt.Scale(domain=[-10, 60]), sort='y'),
        alt.Y('height')
    ) 
        +
    alt.Chart(radar_winds_df).transform_filter(
        alt.datum.height <= 200
    ).transform_filter(
        alt.datum.wind_speed_error <= 50
    ).mark_line().encode(
        alt.X('wind_speed_lower_bound:Q'),
        alt.X2('wind_speed_upper_bound:Q'),
        alt.Y('height:Q')
    ) 

).properties(width=100).facet(
    column=alt.Column(
        'time:T', 
        header=alt.Header(format="%m %d %H:%M")
    )
)

In [ ]:
radar_wind_direction_profile_plots = (
    alt.Chart(radar_winds_df).transform_filter(
        alt.datum.height <= 200
    ).transform_filter(
        alt.datum.wind_speed_error <= 50
    ).mark_line().encode(
        alt.X('wind_direction', title='Wind Direction',
         scale=alt.Scale(domain=[-180, 180], nice=False), 
         axis=alt.Axis(
                values=[-180, -90, 0, 90, 180],
            ),
        sort='y'
        ),
        alt.Y('height')
    ) 
).properties(width=100).facet(
    column=alt.Column(
        'time:T', 
        header=alt.Header(format="%m %d %H:%M")
    )
)

In [ ]:
radar_wind_speed_profile_plots & radar_wind_direction_profile_plots